In [1]:
import json

In [2]:

fileName = "openalexworks.json"
with open(fileName, encoding="utf8") as jsonData:
    openAlexData = json.load(jsonData)

In [3]:
singleEntry = openAlexData[0]
for key in singleEntry:
    print(key)

print(singleEntry["topics"])

id
title
display_name
author
ab
publication_date
so
so_id
host_organization
issn_l
url
pdf_url
version
first_page
last_page
volume
issue
is_oa
is_oa_anywhere
oa_status
oa_url
any_repository_has_fulltext
language
grants
cited_by_count
counts_by_year
publication_year
cited_by_api_url
ids
doi
type
referenced_works
related_works
is_paratext
is_retracted
concepts
topics
doi_woaii
[{'i': 1, 'score': 0.9988, 'name': 'topic', 'id': 'https://openalex.org/T10230', 'display_name': 'Impacts of Climate Change on Marine Fisheries'}, {'i': 1, 'score': 0.9988, 'name': 'subfield', 'id': 'https://openalex.org/subfields/2306', 'display_name': 'Global and Planetary Change'}, {'i': 1, 'score': 0.9988, 'name': 'field', 'id': 'https://openalex.org/fields/23', 'display_name': 'Environmental Science'}, {'i': 1, 'score': 0.9988, 'name': 'domain', 'id': 'https://openalex.org/domains/3', 'display_name': 'Physical Sciences'}, {'i': 2, 'score': 0.9951, 'name': 'topic', 'id': 'https://openalex.org/T11088', 'display_

In [67]:
class Node:
    def __init__(self, name, openAlexURL, displayName, paper):
        self.name = name
        self.openAlexURL = openAlexURL
        self.displayName = displayName
        if(displayName == "Social Sciences"):
            self.displayName = displayName + "(" +self.name + ")"
        self.children = []
        self.paper = paper
    def addChild(self, child):
        self.children.append(child)
    def toText(self):
        text = self.displayName
        if(len(self.children) != 0):
            text += "->"
        for child in self.children:
            text += child.toText()
        return text
    def convertToArray(self):
        temp = []
        
        
        if(len(self.children) != 0):
            temp.append(self.displayName)
            childArray = self.children[0].convertToArray()
            for child in childArray:
                temp.append(child)
            return temp
        else:
            temp.append(self.displayName)
            #temp.append(self.paper)
            return temp
            


In [68]:


def processTopic(topics): #array of the topics
    topNodes = []
    previousNode = None
    currentNode = None
    for topic in topics:
        currentNode = Node(topic['name'], topic['id'], topic['display_name'], topic['id'])
        if currentNode.name != 'topic':
            currentNode.addChild(previousNode)
        if currentNode.name == 'domain':
            topNodes.append(currentNode)
        previousNode = currentNode
    return topNodes



In [69]:
allTopics = []
    
for entry in openAlexData:

    results = processTopic(entry["topics"])
    for result in results:
        allTopics.append(result)


In [70]:
print(len(allTopics))
print(allTopics[0].toText())


allArrays = []
for topic in allTopics:
    allArrays.append(topic.convertToArray())


9944
Physical Sciences->Environmental Science->Global and Planetary Change->Impacts of Climate Change on Marine Fisheries


In [71]:
#print(allArrays[0])
topicDict = {}
topicDict['All Topics'] = ''
for line in allArrays:
    for i in range(len(line)-1, -1, -1):
        if(i == 0):
            topicDict[line[i]] = 'All Topics'
            
        else:
            if line[i] in topicDict:
                if line[i-1] != topicDict[line[i]]:
                    print("Parent Error. Topic: " + line[i] + " has two parents(" + line[i-1] + " | " + topicDict[line[i]] +")")
                    print(line)
                    print(' ')
            elif(line[i] == "Industrial and Manufacturing Engineering"):
                temp = line[i] + " (" + line[i-1] + ")"
                topicDict[temp] = line[i-1]
            elif(line[i] == 'Pharmacology'):
                temp = line[i] + " (" + line[i-1] + ")"
                topicDict[temp] = line[i-1]
            elif(line[i] == 'Physiology'):
                temp = line[i] + " (" + line[i-1] + ")"
                topicDict[temp] = line[i-1]

            else:
                topicDict[line[i]] = line[i-1]


In [72]:
print(topicDict['All Topics'])
print(len(topicDict))
print(topicDict['Social Sciences'])


693


KeyError: 'Social Sciences'

In [73]:
import csv

In [74]:
with open('hierachy.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter='|')
    writer.writerow(['name'] + ['parent'])
    for key in topicDict:
        writer.writerow([key] + [topicDict[key]])

In [ ]:
#with open('hierachy.csv', 'w', newline='') as csvfile:
 #   writer = csv.writer(csvfile, delimiter=',')
  #  writer.writerow(['name'] + ['parent'])
   # writer.writerow(['All Topics'] + [''])
    
   # for line in allArrays:
    #    for i in range(len(line)-1, -1, -1):
     #     if(i == 0):
      #          writer.writerow([line[i]] + ['All Topics'])
       #   else:
        #        writer.writerow([line[i]] + [line[i-1]])
